In [22]:
import requests 
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

In [23]:

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Q1. Running Elastic
Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:
```bash
curl localhost:9200
```
What's the version.build_hash value?

In [25]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info().get('version').get('build_hash')

'42f05b9372a9a4a470db3b52817899b99a76ee73'

# Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:
```bash
pip install elasticsearch
```
Which function do you use for adding your data to elastic?
**index**

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            },
            "section": {
                "type": "text"
            },
            "question":{
                "type": "text"
            },
            "course": {
                "type": "keyword"
            }
        }
    }
}

index_name = "course-questions"

In [27]:
# if already exists, delete it
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [28]:
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [02:25<00:00,  6.53it/s]


# Q3. Searching
Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result? **84.05**

In [29]:
query = "How do I execute a command in a running docker container?"
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [30]:
es_response = es_client.search(index=index_name, body=search_query)

In [31]:
result_docs = [doc['_source'] for doc in es_response['hits']['hits']]
result_docs[0]

{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I debug a docker container?',
 'course': 'machine-learning-zoomcamp'}

In [32]:
result_scores = [doc['_score'] for doc in es_response['hits']['hits']]
result_scores[0]

84.050095

# Q4. Filtering
Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

**How do I copy files from a different folder into docker container’s working directory?**


In [33]:
query = "How do I execute a command in a running docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [34]:
es_response = es_client.search(index=index_name, body=search_query)

In [37]:
result_docs = [doc['_source'] for doc in es_response['hits']['hits']]
result_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

# Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)
    
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```
Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:
    
```python
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```
What's the length of the resulting prompt? (use the len function)

In [38]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [39]:
# torch.set_default_device("cuda")

# # model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto")
# # tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")

# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", torch_dtype="auto")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")        

In [40]:
#generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [41]:
# model_id = "meta-llama/Meta-Llama-3-8B"

# pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")

In [42]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join([context_template.format(**doc) for doc in result_docs])
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [43]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context)
print(prompt)   

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [44]:
# prompt length
len(prompt.strip())

1462

# Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query:

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the encode function. How many tokens does our prompt have?

Note: to decode back a token into a word, you can use the decode_single_token_bytes function:

```python
encoding.decode_single_token_bytes(63842)
```

In [45]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [46]:
len(encoding.encode(prompt))

322

In [47]:
len([encoding.decode_single_token_bytes(token) for token in encoding.encode(prompt)])

322

# Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

In [56]:
import ollama
response = ollama.generate(model='phi3', prompt=prompt)['response']
print(response)

 To execute a command in a running Docker container, you can use the `docker exec` command. Here's how to do it:

If the container is already running, execute a command in the specific container using:
```
docker exec -it <container-id> bash
```

By executing this command with your desired shell or command, the specified Docker container will run that command within its environment.


# Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices here

On June 17, the prices for gpt4o are:

Input: $0.005 / 1K tokens
Output: $0.015 / 1K tokens
You can redo the calculations with the values you got in Q6 and Q7.

In [54]:
avg_input_tokens = 150
avg_output_tokens = 250
n_requests = 1000
price_input = 0.005
price_output = 0.015

input_cost = avg_input_tokens * n_requests * price_input / 1000
output_cost = avg_output_tokens * n_requests * price_output / 1000
print(input_cost , output_cost)

total_cost = input_cost + output_cost
print(total_cost)

0.75 3.75
4.5


In [55]:
# cost of prompt input
prompt_input_cost = len(encoding.encode(prompt)) * price_input / 1000
print(prompt_input_cost)

# cost of response output
response_output_cost = len(response) * price_output / 1000
print(response_output_cost)

0.00161
0.00432
